In [41]:
import json
import gpflow
import numpy as np

from gpras.gpr import RasEmulator
from hecstac.ras.item import RASModelItem

import matplotlib.pyplot as plt
from sklearn.model_selection import KFold


In [11]:
# Constants
ras_path = "/workspaces/gpras/data_dir/Muncie/Muncie.prj"
crs = "ESRI:102673"
stac_path = "/workspaces/gpras/data_dir/Muncie/Muncie.stac.json"
emulator_meta_path = "ras_emulator.json"

In [29]:
# Make emulator metadata json.  Only need to run once.
metadata = {
    "ras_model_path": stac_path,
    "hf_mesh_id": "2D Interior Area",
    "lf_mesh_id": "2D Interior Area",
    "lf_runs": ["lf1", "lf2", "lf3"],
    "hf_runs": ["hf1", "hf2", "hf3"],
    "testing_runs": [],
    "enable_cell_weighting": True
}
with open(emulator_meta_path, mode="w") as f:
    json.dump(metadata, f, indent=4)

In [13]:
# Make initial STAC item. Only need to run once.
stac = RASModelItem.from_prj(ras_path, crs)
with open(stac_path, mode="w") as f:
    json.dump(stac.to_dict(), f, indent=4)

In [106]:
# Start GPR model
import importlib
import gpras.gpr
importlib.reload(gpras.gpr)
from gpras.gpr import RasEmulator
with open(emulator_meta_path) as f:
    emulator_meta = json.load(f)
gpr = RasEmulator(**emulator_meta)
import logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


In [96]:
gpr.mean_across_timesteps

,0,1,2,3,4,5,6,7,8,9,...,5381,5382,5383,5384,5385,5386,5387,5388,5389,5390
run,,,,,,,,,,,,,,,,,,,,,
hf1,942.537415,942.209045,942.061523,942.198853,942.531982,941.897034,942.002625,941.897156,941.596375,941.119263,...,945.966248,945.900391,945.900208,945.957825,945.952332,945.950500,946.070068,946.003235,946.303528,946.117004
hf2,940.460449,939.851868,939.574585,939.832947,940.450256,939.262451,939.463257,939.262451,938.680603,937.724609,...,944.293030,944.080444,944.096985,944.293030,944.287720,944.290466,944.690125,944.497192,945.500549,944.951477
hf3,942.083679,941.741028,941.589966,941.730530,942.077881,941.423767,941.530151,941.423767,941.125671,940.670105,...,945.764893,945.726013,945.722961,945.761902,945.756836,945.757812,945.833069,945.791199,945.985168,945.861389


In [102]:
gpr.mean_across_timesteps

,0,1,2,3,4,5,6,7,8,9,...,5381,5382,5383,5384,5385,5386,5387,5388,5389,5390
run,,,,,,,,,,,,,,,,,,,,,
hf1,942.537415,942.209045,942.061523,942.198853,942.531982,941.897034,942.002625,941.897156,941.596375,941.119263,...,945.966248,945.900391,945.900208,945.957825,945.952332,945.950500,946.070068,946.003235,946.303528,946.117004
hf2,940.460449,939.851868,939.574585,939.832947,940.450256,939.262451,939.463257,939.262451,938.680603,937.724609,...,944.293030,944.080444,944.096985,944.293030,944.287720,944.290466,944.690125,944.497192,945.500549,944.951477
hf3,942.083679,941.741028,941.589966,941.730530,942.077881,941.423767,941.530151,941.423767,941.125671,940.670105,...,945.764893,945.726013,945.722961,945.761902,945.756836,945.757812,945.833069,945.791199,945.985168,945.861389


In [104]:
means_per_run = np.array([gpr.mean_across_timesteps.loc[run, :] for run in gpr.hf_runs])

In [107]:
gpr.n_timesteps_per_run
gpr.mean_across_timesteps
gpr.perform_eof_kfold_analysis(n_splits=3, n_modes=100)

2025-06-24 06:39:11,795 - INFO - Starting EOF analysis across 3 folds with 100 EOF modes...
2025-06-24 06:39:11,796 - INFO - 
--- Fold fold_0 ---
2025-06-24 06:39:11,797 - INFO - Training runs: ['hf2', 'hf3']
2025-06-24 06:39:11,819 - INFO - Unioned wet cells: 4815
2025-06-24 06:39:11,824 - INFO - Computed weighted temporal mean over wet cells
2025-06-24 06:39:12,002 - INFO - Partial fit on run hf2 (289 timesteps)
2025-06-24 06:39:12,329 - INFO - Partial fit on run hf3 (289 timesteps)
2025-06-24 06:39:12,329 - INFO - Selected 100 significant EOFs (North's Rule)
2025-06-24 06:39:12,330 - INFO - EOFs and statistics stored for fold fold_0

2025-06-24 06:39:12,330 - INFO - 
--- Fold fold_1 ---
2025-06-24 06:39:12,331 - INFO - Training runs: ['hf1', 'hf3']
2025-06-24 06:39:12,332 - INFO - Unioned wet cells: 4885
2025-06-24 06:39:12,336 - INFO - Computed weighted temporal mean over wet cells
2025-06-24 06:39:12,472 - INFO - Partial fit on run hf1 (289 timesteps)
2025-06-24 06:39:12,730 - INF

In [63]:
print(gpr.hf_runs)
gpr.generate_kfold_splits(gpr.hf_runs, n_splits = 3, shuffle = True, random_state = 42)

['hf1', 'hf2', 'hf3']


{'fold_0': {'train': ['hf2', 'hf3'], 'test': ['hf1']},
 'fold_1': {'train': ['hf1', 'hf3'], 'test': ['hf2']},
 'fold_2': {'train': ['hf1', 'hf2'], 'test': ['hf3']}}

In [38]:
area_dict, area_gdf = gpr.cell_area_weights

In [23]:
gpr.data_x

/workspaces/gpras/gpras/gpr.py:388: UserWarning: `keep_geom_type=True` in overlay resulted in 85 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  mesh_resampled["area"] = mesh_resampled.geometry.area


0           1           2           3           4     \
run t                                                                 
lf1 0    929.312500  929.312500  929.312500  929.312500  930.625000   
    1    929.312500  929.312500  929.312500  929.312500  930.625000   
    2    929.312500  929.312500  929.312500  929.312500  930.625000   
    3    929.312500  929.312500  929.312500  929.312500  930.625000   
    4    929.312500  929.312500  929.312500  929.312500  930.625000   
...             ...         ...         ...         ...         ...   
lf3 284  945.068237  945.068237  945.068237  945.068237  945.068298   
    285  945.068298  945.068298  945.068298  945.068298  945.068359   
    286  945.068420  945.068420  945.068420  945.068420  945.068420   
    287  945.068481  945.068481  945.068481  945.068481  945.068481   
    288  945.068542  945.068542  945.068542  945.068542  945.068542   

               5           6           7           8           9     ...  \
run t                                                                ...   
lf1 0    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    1    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    2    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    3    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    4    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
...             ...         ...         ...         ...         ...  ...   
lf3 284  945.068237  945.068237  945.068237  945.068237  945.068237  ...   
    285  945.068298  945.068298  945.068298  945.068298  945.068298  ...   
    286  945.068359  945.068359  945.068359  945.068359  945.068359  ...   
    287  945.068420  945.068420  945.068420  945.068420  945.068420  ...   
    288  945.068481  945.068542  945.068542  945.068542  945.068542  ...   

               5381        5382        5383        5384        5385  \
run t                                                                 
lf1 0    941.437500  938.783691  941.437500  940.156982  942.093750   
    1    941.437500  938.783691  941.437500  940.156982  942.093750   
    2    941.437500  938.783691  941.437500  940.156982  942.093750   
    3    941.437500  938.783691  941.437500  940.156982  942.093750   
    4    941.437500  938.783691  941.437500  940.156982  942.093750   
...             ...         ...         ...         ...         ...   
lf3 284  946.292725  946.278687  946.292725  946.248962  946.277710   
    285  946.292969  946.278931  946.292969  946.249207  946.277954   
    286  946.293152  946.279175  946.293152  946.249390  946.278198   
    287  946.293396  946.279419  946.293396  946.249634  946.278381   
    288  946.293640  946.279663  946.293640  946.249878  946.278625   

               5386        5387        5388        5389        5390  
run t                                                                
lf1 0    940.156982  942.093750  940.156982  942.093750  940.156982  
    1    940.156982  942.093750  940.156982  942.093750  940.156982  
    2    940.156982  942.093750  940.156982  942.093750  940.156982  
    3    940.156982  942.093750  940.156982  942.093750  940.156982  
    4    940.156982  942.093750  940.156982  942.093750  940.156982  
...             ...         ...         ...         ...         ...  
lf3 284  946.248962  946.277710  946.248962  946.277710  946.248962  
    285  946.249207  946.277954  946.249207  946.277954  946.249207  
    286  946.249390  946.278198  946.249390  946.278198  946.249390  
    287  946.249634  946.278381  946.249634  946.278381  946.249634  
    288  946.249878  946.278625  946.249878  946.278625  946.249878  

[867 rows x 5391 columns]

In [26]:
classification_qc_shp = "/workspaces/gpras/data_dir/Muncie_qc/classification_qc.shp"
gpr.flood_classification_shp(gpr.hf_runs, classification_qc_shp)

In [47]:
from hecstac.ras.assets import GeometryHdfAsset, PlanHdfAsset
geometry_asset = next(
    asset for asset in gpr.hf_model.assets.values()
    if isinstance(asset, GeometryHdfAsset)
)

ras_geom = geometry_asset.file.hdf_object

mesh_id = "2D Interior Area"
# This is the real source of elevation
mesh_hdf = ras_geom.file["Geometry/2D Flow Areas/" + mesh_id]

# Confirm what keys exist inside this mesh
print(list(mesh_hdf.keys()))  # Should show something like ['Cells Terrain Elevation', 'Cells Center Coordinate']

# Load minimum elevation array
elevations = mesh_hdf["Cells Minimum Elevation"][()]  # shape (n_cells,)

# Preview first 5 entries as a dictionary
preview = {cell_id: elevations[cell_id] for cell_id in range(5)}
print(preview)



['Cells Center Coordinate', "Cells Center Manning's n", 'Cells Face and Orientation Info', 'Cells Face and Orientation Values', 'Cells FacePoint Indexes', 'Cells Minimum Elevation', 'Cells Surface Area', 'Cells Volume Elevation Info', 'Cells Volume Elevation Values', 'FacePoints Cell Index Values', 'FacePoints Cell Info', 'FacePoints Coordinate', 'FacePoints Face and Orientation Info', 'FacePoints Face and Orientation Values', 'FacePoints Is Perimeter', 'Faces Area Elevation Info', 'Faces Area Elevation Values', 'Faces Cell Indexes', 'Faces FacePoint Indexes', 'Faces Low Elevation Centroid', 'Faces Minimum Elevation', 'Faces NormalUnitVector and Length', 'Faces Perimeter Info', 'Faces Perimeter Values', 'Perimeter']
{0: 929.3125, 1: 929.3125, 2: 930.625, 3: 934.2502, 4: 929.0}


In [10]:
from hecstac.ras.utils import find_model_files
find_model_files(ras_path)

['/workspaces/gpras/data_dir/Muncie/Muncie.g03.hdf',
 '/workspaces/gpras/data_dir/Muncie/Muncie.O08',
 '/workspaces/gpras/data_dir/Muncie/Muncie.ic.o06',
 '/workspaces/gpras/data_dir/Muncie/Muncie.r07',
 '/workspaces/gpras/data_dir/Muncie/Muncie.bco07',
 '/workspaces/gpras/data_dir/Muncie/Muncie.bco01',
 '/workspaces/gpras/data_dir/Muncie/Muncie.b03',
 '/workspaces/gpras/data_dir/Muncie/Muncie.g03',
 '/workspaces/gpras/data_dir/Muncie/Muncie.u02.hdf',
 '/workspaces/gpras/data_dir/Muncie/Muncie.p04',
 '/workspaces/gpras/data_dir/Muncie/Muncie.b06',
 '/workspaces/gpras/data_dir/Muncie/Muncie.r05',
 '/workspaces/gpras/data_dir/Muncie/Muncie.r08',
 '/workspaces/gpras/data_dir/Muncie/Muncie.b07',
 '/workspaces/gpras/data_dir/Muncie/Muncie.O04',
 '/workspaces/gpras/data_dir/Muncie/Muncie.p04.hdf',
 '/workspaces/gpras/data_dir/Muncie/Muncie.r06',
 '/workspaces/gpras/data_dir/Muncie/Muncie.x03',
 '/workspaces/gpras/data_dir/Muncie/Muncie.rasmap',
 '/workspaces/gpras/data_dir/Muncie/Muncie.p03.

In [9]:
from hecstac.ras.utils import find_model_files
import os

os.chdir("/workspaces/gpras/")
find_model_files(ras_path)

['/workspaces/gpras/data_dir/Muncie/Muncie.g03.hdf',
 '/workspaces/gpras/data_dir/Muncie/Muncie.O08',
 '/workspaces/gpras/data_dir/Muncie/Muncie.ic.o06',
 '/workspaces/gpras/data_dir/Muncie/Muncie.r07',
 '/workspaces/gpras/data_dir/Muncie/Muncie.bco07',
 '/workspaces/gpras/data_dir/Muncie/Muncie.bco01',
 '/workspaces/gpras/data_dir/Muncie/Muncie.b03',
 '/workspaces/gpras/data_dir/Muncie/Muncie.g03',
 '/workspaces/gpras/data_dir/Muncie/Muncie.u02.hdf',
 '/workspaces/gpras/data_dir/Muncie/Muncie.p04',
 '/workspaces/gpras/data_dir/Muncie/Muncie.b06',
 '/workspaces/gpras/data_dir/Muncie/Muncie.r05',
 '/workspaces/gpras/data_dir/Muncie/Muncie.r08',
 '/workspaces/gpras/data_dir/Muncie/Muncie.b07',
 '/workspaces/gpras/data_dir/Muncie/Muncie.O04',
 '/workspaces/gpras/data_dir/Muncie/Muncie.p04.hdf',
 '/workspaces/gpras/data_dir/Muncie/Muncie.r06',
 '/workspaces/gpras/data_dir/Muncie/Muncie.x03',
 '/workspaces/gpras/data_dir/Muncie/Muncie.rasmap',
 '/workspaces/gpras/data_dir/Muncie/Muncie.p03.

In [11]:
gpr.data_y



0           1           2           3           4     \
run t                                                                 
0   0    940.156250  939.428467  939.091248  939.405579  940.144287   
    1    940.156250  939.428467  939.091248  939.405579  940.144287   
    2    940.156250  939.428467  939.091248  939.405579  940.144287   
    3    940.156250  939.428467  939.091248  939.405579  940.144287   
    4    940.156250  939.428467  939.091248  939.405579  940.144287   
...             ...         ...         ...         ...         ...   
2   284  945.061401  945.061401  945.061401  945.061401  945.061401   
    285  945.061462  945.061462  945.061523  945.061523  945.061523   
    286  945.061523  945.061584  945.061584  945.061584  945.061584   
    287  945.061646  945.061646  945.061646  945.061646  945.061646   
    288  945.061707  945.061707  945.061707  945.061707  945.061707   

               5           6           7           8           9     ...  \
run t                                                                ...   
0   0    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    1    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    2    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    3    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    4    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
...             ...         ...         ...         ...         ...  ...   
2   284  945.061401  945.061401  945.061401  945.061401  945.061401  ...   
    285  945.061462  945.061462  945.061462  945.061462  945.061462  ...   
    286  945.061523  945.061523  945.061523  945.061523  945.061523  ...   
    287  945.061584  945.061584  945.061584  945.061584  945.061584  ...   
    288  945.061646  945.061646  945.061646  945.061646  945.061646  ...   

               5381        5382        5383        5384        5385  \
run t                                                                 
0   0    943.062500  942.710205  942.750000  943.062500  943.062500   
    1    943.062500  942.710205  942.750000  943.062500  943.062500   
    2    943.062500  942.710205  942.750000  943.062500  943.062500   
    3    943.062500  942.710205  942.750000  943.062500  943.062500   
    4    943.062500  942.710205  942.750000  943.062500  943.062500   
...             ...         ...         ...         ...         ...   
2   284  946.253662  946.249023  946.250977  946.245178  946.247314   
    285  946.253906  946.249268  946.251160  946.245422  946.247498   
    286  946.254150  946.249512  946.251404  946.245667  946.247742   
    287  946.254395  946.249756  946.251648  946.245850  946.247986   
    288  946.254578  946.249939  946.251892  946.246094  946.248169   

               5386        5387        5388        5389        5390  
run t                                                                
0   0    943.062500  943.718750  943.406250  945.031250  944.161865  
    1    943.062500  943.718750  943.406250  945.031250  944.161865  
    2    943.062500  943.718750  943.406250  945.031250  944.161865  
    3    943.062500  943.718750  943.406250  945.031250  944.161865  
    4    943.062500  943.718750  943.406250  945.031250  944.161865  
...             ...         ...         ...         ...         ...  
2   284  946.240173  946.243164  946.232788  946.221924  946.210327  
    285  946.240417  946.243408  946.232971  946.222107  946.210510  
    286  946.240601  946.243652  946.233215  946.222351  946.210754  
    287  946.240845  946.243835  946.233459  946.222534  946.210938  
    288  946.241089  946.244080  946.233643  946.222778  946.211182  

[867 rows x 5391 columns]

In [10]:
gpr.mean_across_timesteps

,0,1,2,3,4,5,6,7,8,9,...,5381,5382,5383,5384,5385,5386,5387,5388,5389,5390
run,,,,,,,,,,,,,,,,,,,,,
0,942.537415,942.209045,942.061523,942.198853,942.531982,941.897034,942.002625,941.897156,941.596375,941.119263,...,945.966248,945.900391,945.900208,945.957825,945.952332,945.950500,946.070068,946.003235,946.303528,946.117004
1,940.460449,939.851868,939.574585,939.832947,940.450256,939.262451,939.463257,939.262451,938.680603,937.724609,...,944.293030,944.080444,944.096985,944.293030,944.287720,944.290466,944.690125,944.497192,945.500549,944.951477
2,942.083679,941.741028,941.589966,941.730530,942.077881,941.423767,941.530151,941.423767,941.125671,940.670105,...,945.764893,945.726013,945.722961,945.761902,945.756836,945.757812,945.833069,945.791199,945.985168,945.861389


In [ ]:
# Start GPR model
with open(emulator_meta_path) as f:
    emulator_meta = json.load(f)
gpr = RasEmulator(**emulator_meta)

In [127]:
gpr.data_x

/workspaces/gpras/gpras/gpr.py:266: UserWarning: `keep_geom_type=True` in overlay resulted in 85 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  mesh_resampled = gpd.overlay(self.geometry_hf, self.geometry_lf[["cell_id", "geometry"]], how="intersection")


0           1           2           3           4     \
run t                                                                 
0   0    929.312500  929.312500  929.312500  929.312500  930.625000   
    1    929.312500  929.312500  929.312500  929.312500  930.625000   
    2    929.312500  929.312500  929.312500  929.312500  930.625000   
    3    929.312500  929.312500  929.312500  929.312500  930.625000   
    4    929.312500  929.312500  929.312500  929.312500  930.625000   
...             ...         ...         ...         ...         ...   
2   284  945.068237  945.068237  945.068237  945.068237  945.068298   
    285  945.068298  945.068298  945.068298  945.068298  945.068359   
    286  945.068420  945.068420  945.068420  945.068420  945.068420   
    287  945.068481  945.068481  945.068481  945.068481  945.068481   
    288  945.068542  945.068542  945.068542  945.068542  945.068542   

               5           6           7           8           9     ...  \
run t                                                                ...   
0   0    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    1    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    2    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    3    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    4    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
...             ...         ...         ...         ...         ...  ...   
2   284  945.068237  945.068237  945.068237  945.068237  945.068237  ...   
    285  945.068298  945.068298  945.068298  945.068298  945.068298  ...   
    286  945.068359  945.068359  945.068359  945.068359  945.068359  ...   
    287  945.068420  945.068420  945.068420  945.068420  945.068420  ...   
    288  945.068481  945.068542  945.068542  945.068542  945.068542  ...   

               5381        5382        5383        5384        5385  \
run t                                                                 
0   0    941.437500  938.783691  941.437500  940.156982  942.093750   
    1    941.437500  938.783691  941.437500  940.156982  942.093750   
    2    941.437500  938.783691  941.437500  940.156982  942.093750   
    3    941.437500  938.783691  941.437500  940.156982  942.093750   
    4    941.437500  938.783691  941.437500  940.156982  942.093750   
...             ...         ...         ...         ...         ...   
2   284  946.292725  946.278687  946.292725  946.248962  946.277710   
    285  946.292969  946.278931  946.292969  946.249207  946.277954   
    286  946.293152  946.279175  946.293152  946.249390  946.278198   
    287  946.293396  946.279419  946.293396  946.249634  946.278381   
    288  946.293640  946.279663  946.293640  946.249878  946.278625   

               5386        5387        5388        5389        5390  
run t                                                                
0   0    940.156982  942.093750  940.156982  942.093750  940.156982  
    1    940.156982  942.093750  940.156982  942.093750  940.156982  
    2    940.156982  942.093750  940.156982  942.093750  940.156982  
    3    940.156982  942.093750  940.156982  942.093750  940.156982  
    4    940.156982  942.093750  940.156982  942.093750  940.156982  
...             ...         ...         ...         ...         ...  
2   284  946.248962  946.277710  946.248962  946.277710  946.248962  
    285  946.249207  946.277954  946.249207  946.277954  946.249207  
    286  946.249390  946.278198  946.249390  946.278198  946.249390  
    287  946.249634  946.278381  946.249634  946.278381  946.249634  
    288  946.249878  946.278625  946.249878  946.278625  946.249878  

[867 rows x 5391 columns]

In [128]:
gpr.data_y

0           1           2           3           4     \
run t                                                                 
0   0    940.156250  939.428467  939.091248  939.405579  940.144287   
    1    940.156250  939.428467  939.091248  939.405579  940.144287   
    2    940.156250  939.428467  939.091248  939.405579  940.144287   
    3    940.156250  939.428467  939.091248  939.405579  940.144287   
    4    940.156250  939.428467  939.091248  939.405579  940.144287   
...             ...         ...         ...         ...         ...   
2   284  945.061401  945.061401  945.061401  945.061401  945.061401   
    285  945.061462  945.061462  945.061523  945.061523  945.061523   
    286  945.061523  945.061584  945.061584  945.061584  945.061584   
    287  945.061646  945.061646  945.061646  945.061646  945.061646   
    288  945.061707  945.061707  945.061707  945.061707  945.061707   

               5           6           7           8           9     ...  \
run t                                                                ...   
0   0    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    1    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    2    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    3    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    4    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
...             ...         ...         ...         ...         ...  ...   
2   284  945.061401  945.061401  945.061401  945.061401  945.061401  ...   
    285  945.061462  945.061462  945.061462  945.061462  945.061462  ...   
    286  945.061523  945.061523  945.061523  945.061523  945.061523  ...   
    287  945.061584  945.061584  945.061584  945.061584  945.061584  ...   
    288  945.061646  945.061646  945.061646  945.061646  945.061646  ...   

               5381        5382        5383        5384        5385  \
run t                                                                 
0   0    943.062500  942.710205  942.750000  943.062500  943.062500   
    1    943.062500  942.710205  942.750000  943.062500  943.062500   
    2    943.062500  942.710205  942.750000  943.062500  943.062500   
    3    943.062500  942.710205  942.750000  943.062500  943.062500   
    4    943.062500  942.710205  942.750000  943.062500  943.062500   
...             ...         ...         ...         ...         ...   
2   284  946.253662  946.249023  946.250977  946.245178  946.247314   
    285  946.253906  946.249268  946.251160  946.245422  946.247498   
    286  946.254150  946.249512  946.251404  946.245667  946.247742   
    287  946.254395  946.249756  946.251648  946.245850  946.247986   
    288  946.254578  946.249939  946.251892  946.246094  946.248169   

               5386        5387        5388        5389        5390  
run t                                                                
0   0    943.062500  943.718750  943.406250  945.031250  944.161865  
    1    943.062500  943.718750  943.406250  945.031250  944.161865  
    2    943.062500  943.718750  943.406250  945.031250  944.161865  
    3    943.062500  943.718750  943.406250  945.031250  944.161865  
    4    943.062500  943.718750  943.406250  945.031250  944.161865  
...             ...         ...         ...         ...         ...  
2   284  946.240173  946.243164  946.232788  946.221924  946.210327  
    285  946.240417  946.243408  946.232971  946.222107  946.210510  
    286  946.240601  946.243652  946.233215  946.222351  946.210754  
    287  946.240845  946.243835  946.233459  946.222534  946.210938  
    288  946.241089  946.244080  946.233643  946.222778  946.211182  

[867 rows x 5391 columns]

In [60]:
gpr.shared_geometry


AttributeError: 'RasEmulator' object has no attribute 'shared_geometry'

In [ ]:
# Pre-processing RAS model output to identify wet cells for each RAS run
# Loop through the model output for each run
# Is gpr.data_x the HF model output?
AD_idx_all = []
wet_idx_all = []
AF_idx_all = []
TF_idx_all = []
wse_data_mean = []
n_timesteps_per_run = []
for run in hf_runs:
    # wse_data = Model output for RAS run for Run #
    # Compute mean for each cell for each run
    wse_data_mean.append(np.mean(wse_data, axis=0))
    # Classify cells
    AD_idx, wet_idx, AF_idx, TF_idx  = gpr.identify_wetcells
    n_timesteps_in_file.append(wse_data.shape[0])
    AD_idx_all.append(AD_idx)
    wet_idx_all.append(wet_idx)
    AF_idx_all.append(AF_idx)
    TF_idx_all.append(TF_idx)

In [ ]:
# Pre-processing for EOF analysis
for run in hf_runs:
    # Identify indices associated with training data set for each run
    # grp_train_indices
    
    AD_idx_all_grp = [AD_idx_all[i] for i in grp_train_idx]
    AF_idx_all_grp = [AF_idx_all[i] for i in grp_train_idx]
    
    # Find unique indices
    #AD_idx, TF_idx, AF_idx, wet_idx = get_unique_idx_categories(AD_idx_all_grp,AF_idx_all_grp, n_cells)
    
    # Temporal mean  
    #wse_data_mean_train = np.average(wse_data_mean[grp_train_idx,:], axis=0, weights=n_timesteps_in_file[grp_train_idx])
    #ext_data_mean_train = np.average(ext_data_mean[grp_train_idx,:], axis=0, weights=n_timesteps_in_file[grp_train_idx])  
 
    # Run EOF analysis for LF data
    
    